In [1]:
import pandas as pd
from openpyxl import load_workbook
import os
import random
from datetime import datetime, timedelta


# Caminho do arquivo
arquivo = 'dados.xlsx'

# Lê todas as abas como dicionário {nome_da_aba: dataframe}
abas = pd.read_excel(arquivo, sheet_name=None)


# Atribui explicitamente cada aba à variável correspondente
clientes = abas.get('clientes')
maquinas = abas.get('maquinas')
tipo_manutencao = abas.get('tipo_manutencao')
prioridades = abas.get('prioridades')
datas = abas.get('datas')

# display(clientes)
# display(maquinas)
# display(tipo_manutencao)
# display(prioridades)
# display(datas)

def append_chamados(novos_chamados, path_excel='chamados.xlsx'):
    try:
        # Pega o número de linhas do arquivo já existente para não sobrescrever
        book = load_workbook(path_excel)
        sheet = book.active
        max_row = sheet.max_row

        # Abre ExcelWriter no modo append (append + overlay sheet)
        with pd.ExcelWriter(path_excel, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # Escreve sem header, começando na próxima linha disponível
            novos_chamados.to_excel(writer, index=False, header=False, startrow=max_row)

    except FileNotFoundError:
        # Se o arquivo não existe, cria com header
        novos_chamados.to_excel(path_excel, index=False, header=True)

def simular_chamados(qtd, clientes, maquinas, tipo_manutencao, prioridades, datas, path_excel='chamados.xlsx'):
    import pandas as pd
    
    # Verifica se o arquivo existe
    if os.path.exists(path_excel):
        chamados_existentes = pd.read_excel(path_excel)
        ultimo_id = chamados_existentes['id'].max()
        if pd.isna(ultimo_id):
            ultimo_id = 0
    else:
        # Arquivo não existe: cria um DataFrame vazio com as colunas necessárias e salva o arquivo com cabeçalho
        colunas = ['id', 'cliente', 'maquina', 'tipo_manutencao', 'tempo_necessario', 'prioridade', 'data_abertura']
        df_vazio = pd.DataFrame(columns=colunas)
        df_vazio.to_excel(path_excel, index=False)
        ultimo_id = 0
    
    chamados = []
    data_inicio = pd.to_datetime(datas['data_inicio'].iloc[0])
    data_fim = pd.to_datetime(datas['data_fim'].iloc[0])
    delta_dias = (data_fim - data_inicio).days

    for i in range(1, qtd + 1):
        cliente = random.choice(clientes['clientes'].tolist())
        maquina = random.choice(maquinas['maquinas'].tolist())
        tipo_row = tipo_manutencao.sample(1).iloc[0]
        tipo = tipo_row['tipo_manutencao']
        tempo = tipo_row['tempo_necessario']
        prioridade = random.choice(prioridades['prioridade'].tolist())
        data_abertura = (data_inicio + timedelta(days=random.randint(0, delta_dias))).strftime('%Y-%m-%d')

        chamados.append({
            'id': ultimo_id + i,
            'cliente': cliente,
            'maquina': maquina,
            'tipo_manutencao': tipo,
            'tempo_necessario': tempo,
            'prioridade': prioridade,
            'data_abertura': data_abertura
        })

    novos_chamados = pd.DataFrame(chamados)

    append_chamados(novos_chamados, path_excel)

    return novos_chamados


novos_chamados = simular_chamados(
    qtd=200,
    clientes=clientes,
    maquinas=maquinas,
    tipo_manutencao=tipo_manutencao,
    prioridades=prioridades,
    datas=datas,
    path_excel='chamados.xlsx'
)

# display(novos_chamados)

chamados = pd.read_excel('chamados.xlsx')

display(chamados)

,id,cliente,maquina,tipo_manutencao,tempo_necessario,prioridade,data_abertura
0,1,cliente_28,maquina_10,manutencao_preventiva,2,prioridade_1,2025-04-15
1,2,cliente_17,maquina_10,manutencao_corretiva,3,prioridade_3,2025-04-09
2,3,cliente_27,maquina_8,manutencao_corretiva,3,prioridade_3,2025-02-14
3,4,cliente_8,maquina_10,avaliacao_do_equipamento,1,prioridade_2,2025-01-16
4,5,cliente_13,maquina_8,avaliacao_do_equipamento,1,prioridade_1,2025-02-09
...,...,...,...,...,...,...,...
195,196,cliente_15,maquina_8,manutencao_corretiva,3,prioridade_3,2025-05-29
196,197,cliente_29,maquina_5,avaliacao_do_equipamento,1,prioridade_1,2025-03-18
197,198,cliente_28,maquina_4,avaliacao_do_equipamento,1,prioridade_3,2025-04-26
198,199,cliente_3,maquina_4,avaliacao_do_equipamento,1,prioridade_3,2025-01-15
